<a href="https://colab.research.google.com/github/Jamaleldin/MRNET-for-Knee-Diagnosis/blob/master/MRNet_models_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **All needed library imports**

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd

# **Importing the needed notebooks**

In [2]:
%cd '/content/drive/My Drive/Colab Notebooks'

/content/drive/My Drive/Colab Notebooks


In [3]:
!pip install import-ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=96c281431bc3bd4ad2ab673c6504e77e0cbe85e95fdc5832ea61e01959d2782b
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [4]:
import import_ipynb
# importing the needed notebooks
import MRnet_data_extraction as data

importing Jupyter notebook from MRnet_data_extraction.ipynb
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
unzip:  cannot find or open /content/drive/My Drive/MRNET data set/MRNet-v1.0.zip, /content/drive/My Drive/MRNET data set/MRNet-v1.0.zip.zip or /content/drive/My Drive/MRNET data set/MRNet-v1.0.zip.ZIP.


# **Training functions:**

In [0]:
def train_extractor(model:Model,model_type = data.vgg,series = data.axial,anomaly = data.abnormal):
  # obtaining the data
  x,y = data.get_data_train(series,anomaly);
  input_train,output_train,input_validate,output_validate = data.split(x,y);
  # processing to train for the extractor specifically: obtaining only one image from each patient
  input_train = get_one_image(input_train);
  input_validate = get_one_image(input_validate);
  # normalizing the input data
  input_train = data.normalize(input_train);
  # defining the call backs for training
  save_path = data.path_model + data.delim + model_type + data.delim + data.extractor + data.delim+series + '_' + anomaly + data.extension_model;
  save_best = ModelCheckpoint(save_path, monitor='val_acc', mode='max', verbose=2, save_best_only=True);
  stop = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=5);
  # performing a traininig operation with a batch size to overcome any overfitting
  training_history = model.fit(x = input_train, y = output_train,  validation_data=(input_validate,output_validate), batch_size = 20, epochs=50, callbacks=[save_best,stop]);
  # plotting the graph for the training history
  plot(training_history);
  return training_history;

In [0]:
def get_one_image(input_train):
  new_input_train = [];
  for curr in input_train:
     new_input_train.append(curr[0]);
  return new_input_train;

# **Functions for model evaluation:**

In [0]:
def plot(history):
  # drawing the model graph
  pd.DataFrame(history.history).plot(figsize=(10, 7));
  plt.grid(True);
  plt.gca().set_ylim(0, 1);
  plt.show();

In [0]:
def test_extractor(model:Model,series = data.axial,anomaly = data.abnormal):
  # getting test data
  input_test,output_test = data.get_data_test(series,anomaly);
  # processing it to test the extractor specifically: getting one image from each patient only
  input_test = get_one_image(input_test);
  return model.evaluate(input_test,output_test);

# **Functions to manipualte saved models**

In [0]:
def load_model(model_type = data.vgg,series = data.axial,anomaly = data.abnormal,model_part = data.extractor):
  model_path = data.path_model + data.delim + model_type + data.delim + model_part + data.delim+series + '_' + anomaly + data.extension_model;
  return tf.keras.models.load_model(model_path);